In [ ]:
!pip uninstall keras -y
!pip install git+https://github.com/qubvel/segmentation_models
!git clone https://github.com/SlinkoIgor/ImageDataAugmentor.git
    

# training images}

In [ ]:
import nibabel as nib
import skimage.io as io
import numpy as np
img_tr=nib.load('/kaggle/input/segmentation-data/tr_im.nii')
img_tr_arr=img_tr.get_fdata()
img_tr_arr=np.squeeze(img_tr_arr)
img_tr_arr.shape

# Training masks

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(img_tr_arr[:,:,12])

In [ ]:
import nibabel as nib
import skimage.io as io
import numpy as np
img_mask=nib.load('/kaggle/input/segmentation-data/tr_mask.nii')
img_mask_arr=img_mask.get_fdata()
img_mask_arr=np.squeeze(img_mask_arr)
img_mask_arr.shape

In [ ]:
plt.imshow(img_mask_arr[:,:,12])

In [ ]:
import json

In [ ]:
j = open('/kaggle/input/jsonnew/959830.json')
data = json.load(j)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os 
import numpy as np
import pandas as pd

In [ ]:
prefix = '/kaggle/input/covid-segmentation/'

images_radiopedia = np.load(os.path.join(prefix, 'images_radiopedia.npy')).astype(np.float32)
masks_radiopedia = np.load(os.path.join(prefix, 'masks_radiopedia.npy')).astype(np.int8)
images_medseg = np.load(os.path.join(prefix, 'images_medseg.npy')).astype(np.float32)
masks_medseg = np.load(os.path.join(prefix, 'masks_medseg.npy')).astype(np.int8)

test_images_medseg = np.load(os.path.join(prefix, 'test_images_medseg.npy')).astype(np.float32)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def visualize(image_batch, mask_batch=None, pred_batch=None, num_samples=2):
    num_classes = mask_batch.shape[-1] if mask_batch is not None else 0
    fix, ax = plt.subplots(num_classes + 1, num_samples, figsize=(num_samples * 2, (num_classes + 1) * 2))

    for i in range(num_samples):
        ax_image = ax[0, i] if num_classes > 0 else ax[i]
        ax_image.imshow(image_batch[i,:,:,0], cmap='Greys')
        ax_image.set_xticks([]) 
        ax_image.set_yticks([])
        
        if mask_batch is not None:
            for j in range(num_classes):
                if pred_batch is None:
                    mask_to_show = mask_batch[i,:,:,j]
                else:
                    mask_to_show = np.zeros(shape=(*mask_batch.shape[1:-1], 3)) 
                    mask_to_show[..., 0] = pred_batch[i,:,:,j] > 0.5
                    mask_to_show[..., 1] = mask_batch[i,:,:,j]
                ax[j + 1, i].imshow(mask_to_show, vmin=0, vmax=1)
                ax[j + 1, i].set_xticks([]) 
                ax[j + 1, i].set_yticks([]) 

    plt.tight_layout()
    plt.show()

### Images from radiopedia are full CT volumes:
Class 0 is "ground glass"<br>
Class 1 is "consolidations"<br>
Class 2 is "lungs other" – it doesn't mean that it is healthy lungs (you don't need to predict this class)<br>
Class 3 is "background" – not lungs (you don't need to predict this class)<br>

In [ ]:
images_radiopedia.shape

In [ ]:
plt.imshow(images_radiopedia[100,:,:,0])

In [ ]:
visualize(images_radiopedia[20:], masks_radiopedia[20:])

### Images from medseg are individual axial slices:
Classes are same as in radiopedia part

### Images from radiopedia are full CT volumes:
Class 0 is "ground glass"<br>
Class 1 is "consolidations"<br>
Class 2 is "lungs other" – it doesn't mean that it is healthy lungs (you don't need to predict this class)<br>
Class 3 is "background" – not lungs (you don't need to predict this class)<br>

In [ ]:
visualize(images_medseg, masks_medseg)

### Test images from medseg are individual axial slices:
You should make predictions for class 0 "ground glass" and class 1 "consolidation"

In [ ]:
visualize(test_images_medseg)

In [ ]:
plt.imshow(test_images_medseg[9,:,:,0])

In [ ]:
def plot_hists(images1, images2=None):
    plt.hist(images1.ravel(), bins=100, density=True, color='b', alpha=1 if images2 is None else 0.5)
    if images2 is not None:
        plt.hist(images2.ravel(), bins=100, density=True, alpha=0.5, color='orange')
    plt.show();

### Plot images hists:
HU (Hounsfield scale) of radiopedia data (blue) vs medseg data (orange):

In [ ]:
plot_hists(images_radiopedia, images_medseg)

HU (Hounsfield scale) of test medseg data (blue) vs medseg data (orange):

In [ ]:
plot_hists(test_images_medseg, images_medseg)

### Preprocess images:

In [ ]:
def preprocess_images(images_arr, mean_std=None):
    images_arr[images_arr > 500] = 500
    images_arr[images_arr < -1500] = -1500
    min_perc, max_perc = np.percentile(images_arr, 5), np.percentile(images_arr, 95)
    images_arr_valid = images_arr[(images_arr > min_perc) & (images_arr < max_perc)]
    mean, std = (images_arr_valid.mean(), images_arr_valid.std()) if mean_std is None else mean_std
    images_arr = (images_arr - mean) / std
    print(f'mean {mean}, std {std}')
    return images_arr, (mean, std)

images_radiopedia, mean_std = preprocess_images(images_radiopedia)
images_medseg, _ = preprocess_images(images_medseg, mean_std)
test_images_medseg, _ = preprocess_images(test_images_medseg, mean_std)

In [ ]:
images_radiopedia.shape

In [ ]:
images_medseg.shape

In [ ]:
test_images_medseg.shape

Normalized values of radiopedia data (blue) vs medseg data (orange):

In [ ]:
plot_hists(images_radiopedia, images_medseg)


Normalized values of test medseg data (blue) vs medseg data (orange):

In [ ]:
plot_hists(test_images_medseg, images_medseg)

## Split train / val:

In [ ]:
val_indexes, train_indexes = list(range(24)), list(range(24, 100))

train_images = np.concatenate((images_medseg[train_indexes], images_radiopedia))
train_masks = np.concatenate((masks_medseg[train_indexes], masks_radiopedia))
val_images = images_medseg[val_indexes]
val_masks = masks_medseg[val_indexes]

batch_size = len(val_masks)

del images_radiopedia
del masks_radiopedia
del images_medseg
del masks_medseg

In [ ]:
train_images.shape

In [ ]:
train_masks.shape

In [ ]:
val_images.shape

In [ ]:
plt.imshow(val_images[0,:,:,0])

In [ ]:
val_masks.shape

In [ ]:
plt.imshow(val_masks[0,:,:,0])

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import os
import time
import pickle
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
import os
from xgboost import XGBClassifier
#os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
import pickle
from datetime import datetime
import cv2
import math
import numpy as np
import os
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import PIL
import torch
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.nn import init
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch import nn
import time, copy, argparse
import multiprocessing
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from torch import FloatTensor
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,recall_score,precision_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
import random
import functools
import joblib
import pickle
import os
import cv2
from glob import glob
from random import shuffle


In [ ]:
def data_prep():
  root_folder = '/kaggle/input/large-covid19-ct-slice-dataset/curated_data/curated_data'
  diction = {}

  

  dir_images = []
  dir_label = []
  
  count_covid = 0
  count_non_covid = 0
    
  for subdir in os.listdir(root_folder)[1:]:
          print('subdir',os.path.join(root_folder, subdir))
          for file_count, file_name in enumerate( sorted(glob(os.path.join(root_folder, subdir)+ '/*'),key=len) ):
              
              img = cv2.imread(file_name)
              img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
              dir_images.append(img)
              dir_label.append(subdir)
              if subdir =='1NonCOVID':
                  count_non_covid +=1
                  if count_non_covid ==1500:
                    break
              elif subdir == '2COVID':
                  count_covid +=1
                  if count_covid == 1500:
                    break
              

  diction_train = {}
  diction_test = {}
  dir_label_shuf_train = []
  dir_images_shuf_train = []
  index_shuf = list(range(len(dir_label)))
  shuffle(index_shuf)

  train_shuf = index_shuf[0:int(len(index_shuf))]
  

  for i in train_shuf:
      dir_label_shuf_train.append(dir_label[i])
      dir_images_shuf_train.append(dir_images[i])    
          
  diction_train['X_tr'] = dir_images_shuf_train
  diction_train['y_tr'] = dir_label_shuf_train



        
        
  print(len(diction_train['X_tr']))
  ##print(diction_train['y_tr'])
  print(diction_train['X_tr'][0].shape)


  

  with open('training.pickle', 'wb') as handle:
      pickle.dump(diction_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

  

  nrows=224
  ncolumns=224
  channels=1

  batch_size=16
  epochs=1
  # Number of classes
  num_cpu = multiprocessing.cpu_count()
  num_classes = 2
  torch.manual_seed(8)
  torch.cuda.manual_seed(8)
  np.random.seed(8)
  random.seed(8)




  device = torch.device("cpu")

  dbfile = open('training.pickle', 'rb')      
  db = pickle.load(dbfile) 
  X_train,y_train = db['X_tr'],db['y_tr']

  X = []
  #X_train=np.reshape(np.array(X_train),[len(X_train),])
  for img in list(range(0,len(X_train))):
    if X_train[img].ndim>=3:
        X.append(np.moveaxis(cv2.resize(X_train[img][:,:,:3], (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
    else:
        smimg= cv2.cvtColor(X_train[img],cv2.COLOR_GRAY2RGB)
        X.append(np.moveaxis(cv2.resize(smimg, (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
    
    if y_train[img]=='2COVID':
        y_train[img]=1
    elif y_train[img]=='1NonCOVID' :
        y_train[img]=0
    else:
        continue

  x = np.array(X)
  y_train = np.array(y_train)


  outputs_all = []
  labels_all = []

  X_train, X_val, y_train, y_val = train_test_split(x, y_train, test_size=0.2, random_state=2)

  #base_model = tf.keras.applications.VGG19(weights = 'imagenet')
  #model = Model(inputs=base_model.input,outputs = base_model.get_layer('block5_pool').output)
  base_model = tf.keras.applications.ResNet50(weights = 'imagenet')
  model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
  #base_model = tf.keras.applications.DenseNet121(weights='imagenet')
  #model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
  #base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet')
  #model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)

  feature_dict={0:[],1:[]}
  feature_dict_val={0:[],1:[]}
  classes=['0','1']
  for j in range(len(X_train)):
    xx = X_train[j].transpose(2,1,0)
    xx = np.expand_dims(xx,axis=0)
  
    feature_dict[y_train[j]].append(model.predict(xx))


  for j in range(len(X_val)):
    xx = X_val[j].transpose(2,1,0)
    xx = np.expand_dims(xx,axis=0)
  
    feature_dict_val[y_val[j]].append(model.predict(xx))


  pickle.dump(feature_dict, open('features_till_flatten.pkl', "wb" ))
  pickle.dump(feature_dict_val, open('features_till_flatten_val.pkl', "wb" ))



  labels = []
  feature_list = []
  feature_dict=pickle.load(open('features_till_flatten.pkl','rb'))
  names = [0,1]
  for cat,features  in feature_dict.items():
      labels.extend([int(cat)] * len(features))
      feature_list.append(features)
  feature_list = np.concatenate(feature_list)
  feature_list1=[]
  for i in range(0,len(feature_list)):
      feature_list1.append(feature_list[i].ravel())
  

  labels2 = []
  feature_list_val = []
  feature_dict_val=pickle.load(open('features_till_flatten_val.pkl','rb'))
  names = [0,1]
  for cat,features  in feature_dict_val.items():
      labels2.extend([int(cat)] * len(features))
      feature_list_val.append(features)
  feature_list_val = np.concatenate(feature_list_val)
  feature_list2=[]
  for i in range(0,len(feature_list_val)):
      feature_list2.append(feature_list_val[i].ravel())

  scaler = StandardScaler().fit(feature_list1)
  rescaledX = scaler.transform(feature_list1)
  rescaledX_validation = scaler.transform(feature_list2)
  
  return rescaledX, rescaledX_validation, labels, labels2 


In [ ]:
rescaledX, rescaledX_validation, labels,labels2 = data_prep()

In [ ]:
root_folder = '/kaggle/input/large-covid19-ct-slice-dataset/curated_data/curated_data'
diction = {}



dir_images = []
dir_label = []

count_covid = 0
count_non_covid = 0

for subdir in os.listdir(root_folder)[1:]:
      print('subdir',os.path.join(root_folder, subdir))
      for file_count, file_name in enumerate( sorted(glob(os.path.join(root_folder, subdir)+ '/*'),key=len) ):

          img = cv2.imread(file_name)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          dir_images.append(img)
          dir_label.append(subdir)
          if subdir =='1NonCOVID':
              count_non_covid +=1
              if count_non_covid ==1500:
                break
          elif subdir == '2COVID':
              count_covid +=1
              if count_covid == 1500:
                break


diction_train = {}
diction_test = {}
dir_label_shuf_train = []
dir_images_shuf_train = []
index_shuf = list(range(len(dir_label)))
shuffle(index_shuf)

train_shuf = index_shuf[0:int(len(index_shuf))]


for i in train_shuf:
  dir_label_shuf_train.append(dir_label[i])
  dir_images_shuf_train.append(dir_images[i])    

diction_train['X_tr'] = dir_images_shuf_train
diction_train['y_tr'] = dir_label_shuf_train





print(len(diction_train['X_tr']))
##print(diction_train['y_tr'])
print(diction_train['X_tr'][0].shape)




with open('training.pickle', 'wb') as handle:
  pickle.dump(diction_train, handle, protocol=pickle.HIGHEST_PROTOCOL)



nrows=224
ncolumns=224
channels=1

batch_size=16
epochs=1
# Number of classes
num_cpu = multiprocessing.cpu_count()
num_classes = 2
torch.manual_seed(8)
torch.cuda.manual_seed(8)
np.random.seed(8)
random.seed(8)




device = torch.device("cpu")

dbfile = open('training.pickle', 'rb')      
db = pickle.load(dbfile) 
X_train,y_train = db['X_tr'],db['y_tr']

X = []
#X_train=np.reshape(np.array(X_train),[len(X_train),])
for img in list(range(0,len(X_train))):
    if X_train[img].ndim>=3:
        X.append(np.moveaxis(cv2.resize(X_train[img][:,:,:3], (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
    else:
        smimg= cv2.cvtColor(X_train[img],cv2.COLOR_GRAY2RGB)
        X.append(np.moveaxis(cv2.resize(smimg, (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))

    if y_train[img]=='2COVID':
        y_train[img]=1
    elif y_train[img]=='1NonCOVID' :
        y_train[img]=0
    else:
        continue

x = np.array(X)
y_train = np.array(y_train)


outputs_all = []
labels_all = []

X_train, X_val, y_train, y_val = train_test_split(x, y_train, test_size=0.2, random_state=2)

#base_model = tf.keras.applications.VGG19(weights = 'imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('block5_pool').output)
base_model = tf.keras.applications.ResNet50(weights = 'imagenet')
model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
#base_model = tf.keras.applications.DenseNet121(weights='imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
#base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)

feature_dict={0:[],1:[]}
feature_dict_val={0:[],1:[]}
classes=['0','1']
for j in range(len(X_train)):
    xx = X_train[j].transpose(2,1,0)
    xx = np.expand_dims(xx,axis=0)

    feature_dict[y_train[j]].append(model.predict(xx))


for j in range(len(X_val)):
    xx = X_val[j].transpose(2,1,0)
    xx = np.expand_dims(xx,axis=0)

    feature_dict_val[y_val[j]].append(model.predict(xx))


pickle.dump(feature_dict, open('features_till_flatten.pkl', "wb" ))
pickle.dump(feature_dict_val, open('features_till_flatten_val.pkl', "wb" ))



labels = []
feature_list = []
feature_dict=pickle.load(open('features_till_flatten.pkl','rb'))
names = [0,1]
for cat,features  in feature_dict.items():
  labels.extend([int(cat)] * len(features))
  feature_list.append(features)
feature_list = np.concatenate(feature_list)
feature_list1=[]
for i in range(0,len(feature_list)):
  feature_list1.append(feature_list[i].ravel())


labels2 = []
feature_list_val = []
feature_dict_val=pickle.load(open('features_till_flatten_val.pkl','rb'))
names = [0,1]
for cat,features  in feature_dict_val.items():
  labels2.extend([int(cat)] * len(features))
  feature_list_val.append(features)
feature_list_val = np.concatenate(feature_list_val)
feature_list2=[]
for i in range(0,len(feature_list_val)):
  feature_list2.append(feature_list_val[i].ravel())

scaler = StandardScaler().fit(feature_list1)
rescaledX = scaler.transform(feature_list1)
rescaledX_validation = scaler.transform(feature_list2)

In [ ]:
train_img = []
for i,j in enumerate(train_images):
    xx = j.transpose(2,0,1)
    train_img.append(xx)
train_img = np.array(train_img)

nrows = 224
ncolumns = 224
X = []
#X_train=np.reshape(np.array(X_train),[len(X_train),])
for img in list(range(0,len(train_img))):
    if train_img[img].ndim>=3:
        X.append(np.moveaxis(cv2.resize(train_img[img][:,:,:3], (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
    else:
        smimg= cv2.cvtColor(train_img[img],cv2.COLOR_GRAY2RGB)
        X.append(np.moveaxis(cv2.resize(smimg, (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))

   

x = np.array(X)
X_train = x


outputs_all = []
labels_all = []



#base_model = tf.keras.applications.VGG19(weights = 'imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('block5_pool').output)
base_model = tf.keras.applications.ResNet50(weights = 'imagenet')
model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
#base_model = tf.keras.applications.DenseNet121(weights='imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
#base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)

feature_dict={0:[]}


for j in range(len(X_train)):
    xx = X_train[j].transpose(2,1,0)
    xx = np.expand_dims(xx,axis=0)

    feature_dict[0].append(model.predict(xx))




pickle.dump(feature_dict, open('features_till_flatten.pkl', "wb" ))





feature_list = []
feature_dict=pickle.load(open('features_till_flatten.pkl','rb'))

for cat,features  in feature_dict.items():
  #labels.extend([int(cat)] * len(features))
  feature_list.append(features)
feature_list = np.concatenate(feature_list)
feature_list1=[]
for i in range(0,len(feature_list)):
  feature_list1.append(feature_list[i].ravel())




scaler = StandardScaler().fit(feature_list1)
rescaledX_train = scaler.transform(feature_list1)




In [ ]:
rescaledX_train.shape

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [ ]:
clf_model = XGBClassifier()

In [ ]:
clf_model.fit(rescaledX,labels)

In [ ]:
preds_train = clf_model.predict(rescaledX_train)

In [ ]:
np.savetxt("results.csv", preds_train, delimiter=",")

In [ ]:
train_images.shape

In [ ]:
import gc
gc.collect()

In [ ]:
preds_train.shape

## Data generator and augmentations:

In [ ]:
import tensorflow

import albumentations
import cv2

SOURCE_SIZE = 512
TARGET_SIZE = 256

train_augs = albumentations.Compose([
    albumentations.Rotate(limit=360, p=0.9, border_mode=cv2.BORDER_REPLICATE),
    albumentations.RandomSizedCrop((int(SOURCE_SIZE * 0.75), SOURCE_SIZE), 
                                   TARGET_SIZE, 
                                   TARGET_SIZE, 
                                   interpolation=cv2.INTER_NEAREST),
    albumentations.HorizontalFlip(p=0.5),

])

val_augs = albumentations.Compose([
    albumentations.Resize(TARGET_SIZE, TARGET_SIZE, interpolation=cv2.INTER_NEAREST)
])

In [ ]:
class Dataset:   
    def __init__(
            self, 
            images, 
            masks,
            augmentations=None
    ):
        self.images = images
        self.masks = masks
        self.augmentations = augmentations
    
    def __getitem__(self, i):
        image = self.images[i]
        mask = self.masks[i]
        
        if self.augmentations:
            sample = self.augmentations(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        return image, mask
        
    def __len__(self):
        return len(self.images)
    
    
class Dataloder(tensorflow.keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        images = []
        masks = []
        for j in range(start, stop):
            image, mask = self.dataset[self.indexes[j]]
            images.append(image)
            masks.append(mask)
        
        images = np.stack(images, axis=0)
        masks = np.stack(masks, axis=0).astype(np.float32)
        
        return (images, masks)
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)
            
train_dataset = Dataset(train_images, train_masks, train_augs)
val_dataset = Dataset(val_images, val_masks, val_augs)

train_dataloader = Dataloder(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = Dataloder(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
assert train_dataloader[0][0].shape == (batch_size, TARGET_SIZE, TARGET_SIZE, 1)
assert train_dataloader[0][1].shape == (batch_size, TARGET_SIZE, TARGET_SIZE, 4)

In [ ]:
visualize(*next(iter(train_dataloader)))
visualize(*next(iter(val_dataloader)))

## Metrics:

In [ ]:
def fscore_glass(y_true, y_pred):
    return sm.metrics.f1_score(y_true[..., 0:1], 
                               y_pred[..., 0:1])
    
def fscore_consolidation(y_true, y_pred):
    return sm.metrics.f1_score(y_true[..., 1:2], 
                               y_pred[..., 1:2])

def fscore_lungs_other(y_true, y_pred):
    return sm.metrics.f1_score(y_true[..., 2:3], 
                               y_pred[..., 2:3])

def fscore_glass_and_consolidation(y_true, y_pred):
    return sm.metrics.f1_score(y_true[..., :2], 
                               y_pred[..., :2])

In [ ]:
from segmentation_models import Unet
import segmentation_models as sm

from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
np.random.seed(0)

base_model = Unet(backbone_name='efficientnetb1',
                  encoder_weights='imagenet',
                  classes=4, 
                  activation='softmax')

model = Sequential([Input(shape=(TARGET_SIZE, TARGET_SIZE, 1)),
                    Conv2D(3, (1, 1)),  # map N channels data to 3 channels
                    base_model])
    
model.compile(Adam(learning_rate=0.001, amsgrad=True),
              loss=sm.losses.categorical_crossentropy,
              metrics=[fscore_glass, fscore_consolidation, fscore_lungs_other, fscore_glass_and_consolidation])

checkpoint_callback = ModelCheckpoint('best_model',
                                      monitor='fscore_glass_and_consolidation',
                                      mode='max',
                                      save_best_only=True)

model.fit(
    train_dataloader,
    steps_per_epoch=len(train_dataloader) * 6,
    epochs=20,
    validation_data=val_dataloader,
    validation_steps=len(val_dataloader),
    callbacks=[checkpoint_callback],
    workers=4)

In [ ]:
TARGET_SIZE

In [ ]:
from segmentation_models import Unet
import segmentation_models as sm

from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
np.random.seed(0)

## Load best model and visualize predicions on val:

In [ ]:
del train_images
del train_masks

In [ ]:
model = tensorflow.keras.models.load_model('best_model/',
                                           compile=False,
                                           custom_objects={
                                                'categorical_crossentropy': sm.losses.categorical_crossentropy,
                                                'fscore_consolidation': fscore_consolidation, 
                                                'fscore_glass': fscore_glass, 
                                                'fscore_lungs_other': fscore_lungs_other,
                                                'fscore_glass_and_consolidation': fscore_glass_and_consolidation})

model.compile(Adam(learning_rate=0.001, amsgrad=True),
              loss=sm.losses.jaccard_loss)

In [ ]:
import gc
gc.collect()

In [ ]:
print(tf.__version__)

In [ ]:
!pip install tensorflow==2.5.0


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:

from keras.utils import get_custom_objects
from keras import backend as K
from keras.layers import Activation

def swish_activation(x):
        return (K.sigmoid(x) * x)

get_custom_objects().update({'swish_activation': Activation(swish_activation)})

In [ ]:
from tensorflow import keras

In [ ]:
from keras.backend import sigmoid
  
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

In [ ]:
import os
from tensorflow.keras.preprocessing import image
from tf.keras.activations import swish

pb_model_dir = "best_model/"
h5_model = "mymodel.h5"

# Loading the Tensorflow Saved Model (PB)
input_shape = (1,256,256,1)

model = tf.keras.models.load_model(pb_model_dir)
model.build(input_shape)
print(model.summary())

# Saving the Model in H5 Format
tf.keras.models.save_model(model, h5_model)

# Loading the H5 Saved Model
loaded_model_from_h5 = tf.keras.models.load_model(h5_model)
print(loaded_model_from_h5.summary())

In [ ]:
input = val_dataloader[0]
image_batch, mask_batch = input

preds = model.predict_on_batch(image_batch)
visualize(image_batch, mask_batch, pred_batch=preds)

# yellow is TP, red is FP, green is FN

## Test preds:

In [ ]:
image_batch = np.stack([val_augs(image=img)['image'] for img in test_images_medseg], axis=0)
test_preds = model.predict_on_batch(image_batch)
test_masks_prediction = test_preds > 0.5
visualize(image_batch, test_masks_prediction, num_samples=len(test_images_medseg))

## Resize prediction to original size:

In [ ]:
plt.imshow(test_images_medseg[2,:,:,0])

In [ ]:
plt.imshow(test_preds[2,:,:,0])

In [ ]:
import scipy
test_masks_prediction_original_size = scipy.ndimage.zoom(test_masks_prediction[..., :-2], (1, 2, 2, 1), order=0)
test_masks_prediction_original_size.shape

In [ ]:
import pandas as pd

pd.DataFrame(
             data=np.stack((np.arange(len(test_masks_prediction_original_size.ravel())), 
                            test_masks_prediction_original_size.ravel().astype(int)),
                            axis=-1), 
             columns=['Id', 'Predicted'])\
.set_index('Id').to_csv('submission.csv')